In [1]:
import sys
from utilities.get_data import get_historical_from_path
from os import listdir
from os.path import isfile, join
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ta
import os
import subprocess
import importlib

# Use the terminal code to import the data from the internet node .\database\sdl_for_quick_analysis.js

In [2]:
def execute_terminal_command(command):
    try:
        result = subprocess.run(command, shell=True, capture_output=True, text=True)
        # Print the standard output of the command
        print(result.stdout)

        # If you want to capture the standard error as well, uncomment the line below
        # print(result.stderr)

        # Check the return code
        if result.returncode == 0:
            print("Command executed successfully.")
        else:
            print(f"Command execution failed with return code: {result.returncode}")
    except subprocess.CalledProcessError as e:
        print(f"Error executing command: {e}")
def install_package_if_needed(package):
    try:
        # Check if the package is installed
        subprocess.run(['node', '-e', 'require.resolve("{}")'.format(package)], check=True, capture_output=True)
        print(f"{package} is already installed.")
    except subprocess.CalledProcessError as e:
        # Package not found, install it
        print(f"{package} is not installed. Installing...")
        try:
            command = f'npm install {package}'
            execute_terminal_command(command)
            print(f"{package} installed successfully.")
        except subprocess.CalledProcessError as e:
            print(f"Error installing {package}: {e}")
def remove_non_csv_files(directory):
    for filename in os.listdir(directory):
        if not filename.endswith(".csv"):
            file_path = os.path.join(directory, filename)
            if os.path.isfile(file_path):
                os.remove(file_path)
                print(f"Removed file: {file_path}")
def get_analyisis_from_window(window=24):
    metric_list = {}
    for coin in df_list:
        try:
            df = df_list[coin]
            df.drop(columns=df.columns.difference(['open','high','low','close','volume']), inplace=True)
            df["rvolume"] = df["volume"] * df["close"]
            df["olhc"] = (df["open"] + df["low"] + df["high"] + df["close"])/4
            df["atr"] = (ta.volatility.average_true_range(high=df['high'], low=df['low'], close=df['close'])/df["close"])*100
            # df["norm"].dropna(inplace=True)
            volatility = df["atr"].mean()
            mean_volume = df["rvolume"].mean()
            std = df["olhc"].std()/df["olhc"].mean()
            perf = (df["close"].iloc[-1] - df["close"].iloc[0]) / df["close"].iloc[0]
            return_vs_vol = perf/volatility
            last_volume = df["rvolume"].iloc[-window:].mean()
            volume_evolution = last_volume / mean_volume
            last_volatility = df.iloc[-window:]["olhc"].std() / df.iloc[-window:]["olhc"].mean()
            last_perf = (df.iloc[-1]["close"] - df.iloc[-window]["close"]) / df.iloc[-window]["close"]
            volatility_evolution = last_volatility / volatility
            last_return_vs_vol = last_perf / last_volatility
            metric_list[coin] = {
                "mean_volume": round(mean_volume),
                "last_volume": round(last_volume),
                "volume_evolution": round(volume_evolution,2),
                "mean_volatility": round(volatility,2),
                "last_volatility": round(last_volatility*100,2),
                "volatility_evolution": round(volatility_evolution*100,2),
                "return": round(perf*100,2),
                "last_return":round(last_perf*100,2),
                "return_vs_vol": round(return_vs_vol*100,2),
                "last_return_vs_vol":round(last_return_vs_vol,2)
                }
        except:
            # print("Error on", coin)
            # raise
            pass

    return pd.DataFrame.from_dict(metric_list, orient='index')

In [3]:
install_package_if_needed("ccxt")

ccxt is already installed.


In [4]:
install_package_if_needed("csv-writer")

csv-writer is already installed.


In [7]:
command = 'node ./database/dl_for_quick_analysis.js'
execute_terminal_command(command)

database/quick_analysis is deleted!

Loading 1/590 coins downloaded
Loading 2/590 coins downloaded
Loading 3/590 coins downloaded
Loading 4/590 coins downloaded
Loading 5/590 coins downloaded
Loading 6/590 coins downloaded
Loading 7/590 coins downloaded
Loading 8/590 coins downloaded
Loading 9/590 coins downloaded
Loading 10/590 coins downloaded
Loading 11/590 coins downloaded
Loading 12/590 coins downloaded
Loading 13/590 coins downloaded
Loading 14/590 coins downloaded
Loading 15/590 coins downloaded
Loading 16/590 coins downloaded
Loading 17/590 coins downloaded
Loading 18/590 coins downloaded
Loading 19/590 coins downloaded
Loading 20/590 coins downloaded
Loading 21/590 coins downloaded
Loading 22/590 coins downloaded
Loading 23/590 coins downloaded
Loading 24/590 coins downloaded
Loading 25/590 coins downloaded
Loading 26/590 coins downloaded
Loading 27/590 coins downloaded
Loading 28/590 coins downloaded
Loading 29/590 coins downloaded
Loading 30/590 coins downloaded
Loading 31/5

In [8]:
directory_path = "./database/quick_analysis"
remove_non_csv_files(directory_path)

In [9]:
mypath = "./database/quick_analysis"
file_list = [f for f in listdir(mypath) if isfile(join(mypath, f))]
df_list = {}
for file in file_list:
    df_list[file[:-9]] = get_historical_from_path(mypath+"/"+file)

print("All data loaded")

All data loaded


In [10]:
df_list["BTC"]

,open,high,low,close,volume
date,,,,,
2023-04-23 22:00:00,27517.83,27642.03,27423.46,27591.73,1270.69427
2023-04-23 23:00:00,27591.73,27650.28,27543.47,27590.60,1212.95687
2023-04-24 00:00:00,27590.59,27871.69,27427.30,27807.49,2569.36143
2023-04-24 01:00:00,27807.50,28000.00,27685.00,27752.48,3284.37771
2023-04-24 02:00:00,27752.47,27827.53,27670.28,27745.17,1361.78446
...,...,...,...,...,...
2023-06-04 09:00:00,27150.81,27249.86,27150.81,27215.89,806.34780
2023-06-04 10:00:00,27215.88,27230.17,27190.34,27221.05,532.53164
2023-06-04 11:00:00,27221.06,27235.67,27175.51,27198.55,939.50915


In [11]:
df_metric = get_analyisis_from_window(12).sort_values(by="last_return", ascending=False)
df_metric.iloc[:30]

/tmp/ipykernel_15314/2290173438.py:52: RuntimeWarning: invalid value encountered in scalar divide
  return_vs_vol = perf/volatility
/tmp/ipykernel_15314/2290173438.py:54: RuntimeWarning: invalid value encountered in scalar divide
  volume_evolution = last_volume / mean_volume
/tmp/ipykernel_15314/2290173438.py:57: RuntimeWarning: invalid value encountered in scalar divide
  volatility_evolution = last_volatility / volatility
/tmp/ipykernel_15314/2290173438.py:58: RuntimeWarning: invalid value encountered in scalar divide
  last_return_vs_vol = last_perf / last_volatility
/tmp/ipykernel_15314/2290173438.py:52: RuntimeWarning: invalid value encountered in scalar divide
  return_vs_vol = perf/volatility
/tmp/ipykernel_15314/2290173438.py:54: RuntimeWarning: invalid value encountered in scalar divide
  volume_evolution = last_volume / mean_volume
/tmp/ipykernel_15314/2290173438.py:57: RuntimeWarning: divide by zero encountered in scalar divide
  volatility_evolution = last_volatility / vol

,mean_volume,last_volume,volume_evolution,mean_volatility,last_volatility,volatility_evolution,return,last_return,return_vs_vol,last_return_vs_vol
JOE-USDT,1415313,4464419,3.15,1.54,2.99,1.94,-6.39,7.19,-4.14,2.41
JOE,319189,644506,2.02,1.50,2.99,2.00,-6.40,7.14,-4.28,2.39
EPX,53282,1754499,32.93,0.86,4.84,5.61,2.52,6.54,2.93,1.35
AKRO,48600,675769,13.90,1.20,4.82,4.01,4.64,6.21,3.86,1.29
CVP,28028,254685,9.09,1.10,2.82,2.57,-5.32,5.74,-4.84,2.03
IMX,220992,217825,0.99,1.27,2.22,1.75,-12.63,4.53,-9.93,2.04
IMX-USDT,1479052,2177014,1.47,1.30,2.19,1.69,-12.75,4.47,-9.83,2.04
DAR,76622,379770,4.96,0.88,2.44,2.76,-16.32,4.32,-18.47,1.77
DAR-USDT,347136,1738873,5.01,0.93,2.38,2.57,-16.47,4.23,-17.79,1.78
FOR,26242,21921,0.84,0.92,1.16,1.26,-3.43,3.58,-3.73,3.09
